In [1]:
import h5py
import numpy as np
from sklearn.utils import shuffle
import pandas as pd
from keras.preprocessing.image import *

np.random.seed(2017)


Using TensorFlow backend.


the ipynb is refer: 
Create the premodel by ResNet50,Xception, InceptionV3, VGG16, VGG19

In [2]:
def one_hot_encode(y):
    l = list()
    for item in y:
        c = [0. for i in range(10)]
        c[item] = 1.
        l.append(c)
    return np.array(l)

def load_data(pretrain_data_file):
    print('load data: ' + pretrain_data_file)
    with h5py.File("models/new/" + pretrain_data_file, 'r') as h:
        X_train = np.array(h['train'])
        X_valid = np.array(h['valid'])
        y_train = np.array(h['label'])
        y_valid = np.array(h['valid_label'])
        X_train, y_train = shuffle(X_train, y_train)
        y_train = one_hot_encode(y_train)
        X_valid, y_valid = shuffle(X_valid, y_valid)
        y_valid = one_hot_encode(y_valid)
        return X_train, y_train, X_valid, y_valid
    
def load_test_data(data_file):
    print('load test data: ' + data_file)
    with h5py.File("models/" + data_file, 'r') as h:
        X_test = np.array(h['test'])
        return X_test
    
from keras.models import *
from keras.layers import *

def make_model(input_shape):

    input_tensor = Input(input_shape)
    x = input_tensor
    x = Dropout(0.5)(x)
    x = Dense(10, activation='softmax')(x)
    model = Model(input_tensor, x)
    #model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
#     x = Dense(10)(x)
#     x = Activation('softmax')(x)
#     model = Model(input_tensor, x)
#     model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#     #model.compile(Adam(lr=1e-3), loss='categorical_crossentropy')
    
    return model
print("Done")

Done


In [3]:
def gen_kaggle_csv(model, X_test,  model_image_size, csv_name):
    y_pred = model.predict(X_test, verbose=1)
    y_pred = y_pred.clip(min=0.005, max=0.995)
    print()
    print(y_pred[:3])

    dir = "/ext/Data/distracted_driver_detection/"

    gen = ImageDataGenerator()
    test_generator = gen.flow_from_directory(dir + "test/", (model_image_size, model_image_size), shuffle=False, 
                                             batch_size=16, class_mode=None)

    l = list()
    for i, fname in enumerate(test_generator.filenames):
        name = fname[fname.rfind('/')+1:]
        l.append( [name, *y_pred[i]] )

    l = np.array(l)
    data = {'img': l[:,0]}
    for i in range(10):
        data["c%d"%i] = l[:,i+1]
    df = pd.DataFrame(data, columns=['img'] + ['c%d'%i for i in range(10)])
    df.head(10)
    df = df.sort_values(by='img')
    df.to_csv(csv_name, index=None, float_format='%.3f')

print("done")

done


parameters

In [4]:
batch_size = 128
epochs = 30

## ResNet50

In [5]:
X_train, y_train, X_valid, y_valid = load_data("bottleneck_ResNet50.h5")
model_resnet = make_model(X_train.shape[1:])
model_resnet.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_valid,y_valid))
model_resnet.save("models/resnet50-model.h5")
print("model save successed")

load data: bottleneck_ResNet50.h5
Train on 20787 samples, validate on 1637 samples
Epoch 1/30
20787/20787 [==============================] - 1s - loss: 14.5259 - acc: 0.0988 - val_loss: 2.4732 - val_acc: 0.1020

In [6]:
X_test = load_test_data("bottleneck_ResNet50_test.h5")
gen_kaggle_csv(model_resnet,  X_test,  224, 'csv/resnet50-pred.csv')

load test data: bottleneck_ResNet50_test.h5
79424/79726 [============================>.] - ETA: 0s 

## Xception

In [7]:
X_train, y_train, X_vaiid, y_valid = load_data("bottleneck_Xception.h5")
model_xception = make_model(X_train.shape[1:])
model_xception.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_valid,y_valid))
model_xception.save("models/xception-model.h5")
print("model save successed")

load data: bottleneck_Xception.h5
Train on 20787 samples, validate on 1637 samples
Epoch 1/30
20787/20787 [==============================] - 0s - loss: 14.4363 - acc: 0.1043 - val_loss: 2.4859 - val_acc: 0.1106
Epoch 2/30
20787/20787 [==============================] - 0s - 

In [8]:
X_test = load_test_data("bottleneck_Xception_test.h5")
gen_kaggle_csv(model_xception,  X_test,  299, 'csv/xception-pred.csv')

load test data: bottleneck_Xception_test.h5
78656/79726 [============================>.] - ETA: 0s 

## InceptionV3

In [9]:
X_train, y_train, X_valid, y_valid = load_data("bottleneck_InceptionV3.h5")
model_inceptionV3 = make_model(X_train.shape[1:])
model_inceptionV3.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_valid,y_valid))
model_inceptionV3.save("models/inceptionV3-model.h5")
print("model save successed")

load data: bottleneck_InceptionV3.h5
Train on 20787 samples, validate on 1637 samples
Epoch 1/30
20787/20787 [==============================] - 0s - loss: 14.6069 - acc: 0.0938 - val_loss: 2.5274 - val_acc: 0.1014
Epoch 2/30
20787/20787 [==============================] - 0s

In [10]:
X_test = load_test_data("bottleneck_InceptionV3_test.h5")
gen_kaggle_csv(model_inceptionV3,  X_test,  299, 'csv/inceptionV3-pred.csv')

load test data: bottleneck_InceptionV3_test.h5
79296/79726 [============================>.] - ETA: 0s 

## VGG16

In [11]:
X_train, y_train, X_valid, y_valid = load_data("bottleneck_VGG16.h5")
model_vgg16 = make_model(X_train.shape[1:])
model_vgg16.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_valid,y_valid))
model_vgg16.save("models/vgg16-model.h5")
print("model save successed")

load data: bottleneck_VGG16.h5
Train on 20787 samples, validate on 1637 samples
Epoch 1/30
20787/20787 [==============================] - 0s - loss: 14.5184 - acc: 0.0992 - val_loss: 2.5056 - val_acc: 0.1173
Epoch 2/30
20787/20787 [==============================] - 0s - loss: 14.5169 - acc: 0.0993 - val_loss: 2.5056 - val_acc: 0.1173

In [12]:
X_test = load_test_data("bottleneck_VGG16_test.h5")
gen_kaggle_csv(model_vgg16,  X_test,  224, 'csv/vgg16-pred.csv')

load test data: bottleneck_VGG16_test.h5
77184/79726 [============================>.] - ETA: 0s 

## VGG19

In [13]:
X_train, y_train, X_valid, y_valid = load_data("bottleneck_VGG19.h5")
model_vgg19 = make_model(X_train.shape[1:])
model_vgg19.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_valid,y_valid))
model_vgg19.save("models/vgg19-model.h5")
print("model save successed")

load data: bottleneck_VGG19.h5
Train on 20787 samples, validate on 1637 samples
Epoch 1/30
20787/20787 [==============================] - 0s - loss: 14.6467 - acc: 0.0913 - val_loss: 2.6482 - val_acc: 0.0971
Epoch 2/30
20787/20787 [==============================] - 0s - loss: 14.6542 - acc: 0.0908 - val_loss: 2.6482 - val_acc: 0.0971

In [14]:
X_test = load_test_data("bottleneck_VGG19_test.h5")
gen_kaggle_csv(model_vgg16,  X_test,  224, 'csv/vgg19-pred.csv')

load test data: bottleneck_VGG19_test.h5
77600/79726 [============================>.] - ETA: 0s

# Combine multi-modles

In [15]:
import h5py
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
np.random.seed(2017)

X_train = []
X_valid = []

#premodels = ["bottleneck_ResNet50.h5", "bottleneck_Xception.h5", "bottleneck_InceptionV3.h5", "bottleneck_VGG16.h5", "bottleneck_VGG19.h5"]
#premodels = ["bottleneck_ResNet50.h5", "bottleneck_InceptionV3.h5",  "bottleneck_VGG19.h5"]
premodels = ["bottleneck_Xception.h5",  "bottleneck_InceptionV3.h5", ]
for filename in premodels:
    print('------------------'+filename)
    with h5py.File("models/" + filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_valid.append(np.array(h['valid']))
        y_train = np.array(h['label'])
        y_valid = np.array(h['valid_label'])

X_train = np.concatenate(X_train, axis=1)
X_valid = np.concatenate(X_valid, axis=1)

X_train, y_train = shuffle(X_train, y_train)
y_train = one_hot_encode(y_train)
X_valid, y_valid = shuffle(X_valid, y_valid)
y_valid = one_hot_encode(y_valid)

------------------bottleneck_Xception.h5
------------------bottleneck_InceptionV3.h5


In [16]:
model_mix = make_model(X_train.shape[1:])
model_mix.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_valid,y_valid))
model_mix.save("models/mixed-model.h5")
print("model save successed")

Train on 20787 samples, validate on 1637 samples
Epoch 1/30
20787/20787 [==============================] - 0s - loss: 1.3356 - acc: 0.5770 - val_loss: 1.3612 - val_acc: 0.5235
Epoch 2/30
20787/20787 [==============================] - 0s - loss: 0.6386 - acc: 0.8344 - val_loss: 1.224

In [17]:
#test_premodels = ["bottleneck_ResNet50_test.h5", "bottleneck_Xception_test.h5", "bottleneck_InceptionV3_test.h5", 
#             "bottleneck_VGG16_test.h5", "bottleneck_VGG19_test.h5"]
#test_premodels = ["bottleneck_ResNet50_test.h5", "bottleneck_InceptionV3_test.h5", "bottleneck_VGG19_test.h5"]
test_premodels = ["bottleneck_Xception_test.h5", "bottleneck_InceptionV3_test.h5",  ]

X_test = []
for filename in test_premodels:
    print('------------------'+filename)
    with h5py.File("models/" + filename, 'r') as h:
        X_test.append(np.array(h['test']))
        
X_test = np.concatenate(X_test, axis=1)
gen_kaggle_csv(model_mix, X_test,  299, 'csv/mixed-pred.csv')

------------------bottleneck_Xception_test.h5
------------------bottleneck_InceptionV3_test.h5
79616/79726 [============================>.] - ETA: 0s 